In [1]:
from __future__ import print_function
import numpy as np
from astropy.io import fits
from marvin.tools import Maps
from tqdm import tqdm

import math as math
import matplotlib.pyplot as plt
import pylab as plb



#from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import scipy as scipy
#from scipy.fft import fft
from scipy import ndimage
from scipy.optimize import curve_fit
import scipy.stats as stats
import math

import copy
import Simpler_Classifier

from importlib import reload 

from hyperopt import hp, fmin, rand, tpe, space_eval
import torch
from torch import nn


[INFO]: No release version set. Setting default to DR17
[WARNING]: path /Users/runquanguan/sas/mangawork/manga/spectro/redux/v3_1_1/drpall-v3_1_1.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /Users/runquanguan/sas/mangawork/manga/spectro/analysis/v3_1_1/3.1.0/dapall-v3_1_1-3.1.0.fits cannot be found. Setting dapall to None. (MarvinUserWarning)
[WARNING]: Passing field metadata as keyword arguments is deprecated. Use the explicit `metadata=...` argument instead. Additional metadata: {'location': 'view_args'} (RemovedInMarshmallow4Warning)
[WARNING]: The 'missing' argument to fields is deprecated. Use 'load_default' instead. (RemovedInMarshmallow4Warning)


In [2]:
# Gaussian fit method

In [3]:
data = '8138-6101'
#data = ‘8597-1901’ # 156 sample

In [4]:
curve = Simpler_Classifier.simple_integ(data)

[WARNING]: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray. (VisibleDeprecationWarning)


In [5]:
def triangle_model_loss(data):
    
    curve = Simpler_Classifier.simple_integ(data)

    def cone_model(h, left, right, cont):

        if left+right >= 36 or left+right <= 2 or h <= 1:
            model = np.linspace(0.1,0.1,36)

        else:

            wid = left + right
            radius = h
            w_res = math.floor((36-wid)/2)
            r_var = np.linspace(0,radius,w_res)

            y1 = []

            circ_area = np.pi * (radius**2)
            for i in r_var:

                l = radius - i
                angle_rad = np.arccos(l/radius)
                w = np.sin(angle_rad)*radius
                triangles = l*w

                angle_res = (np.pi*2) - (angle_rad*2)
                disc_area = (angle_res/(2*np.pi)) * circ_area

                total_area = circ_area - (disc_area + triangles)
                y1.append(total_area)


            y2 = copy.deepcopy(y1)
            y1.reverse()
            y = y2 + y1[1:]

            model = list(np.array(list(np.zeros(left)) + y + list(np.zeros(right))) + cont)

        x = np.linspace(0,len(model),len(model))
        y = model
        x2 = np.linspace(0,len(model),36)
        f_linear = scipy.interpolate.interp1d(x, y, kind='linear')
        intp_model = f_linear(x2)

        return intp_model

    def loss_function_a(arg):

        h, left, right, cont= arg
        onecone = cone_model(h,left, right, cont)
        loss = 0
        for i in range(36):
            loss = loss + abs(curve[i]-onecone[i])

        total_loss = loss.item()

        return total_loss

    def loss_function_b(arg):

        h, left, right, cont= arg
        onecone = cone_model(h,left, right, cont)
        loss = 0
        for i in range(36):
            loss = loss + abs(curve[i+36]-onecone[i])

        total_loss = loss.item()

        return total_loss

    space = [hp.uniform('h',0, 100), hp.randint('left',35), hp.randint('right',35),hp.uniform('cont',0, 3)]
    best_a = fmin(loss_function_a, space, algo=tpe.suggest, max_evals = 2000)
    best_b = fmin(loss_function_b, space, algo=tpe.suggest, max_evals = 2000)
    total_loss = loss_function_a(space_eval(space, best_a))+loss_function_b(space_eval(space, best_b))
    return total_loss

In [61]:
def norm_model_loss(data):
    
    curve = Simpler_Classifier.simple_integ(data)
    
    def cone_norm_model(mu, variance, scale, contin):
        x = np.linspace(0, 36, 36)
        sigma = math.sqrt(variance)
        cone = stats.norm.pdf(x, mu, sigma)*scale
        onecone = cone+contin
        return onecone


    def loss_function_c(arg):

        mu, variance, scale, contin= arg
        onecone = cone_norm_model(mu, variance, scale, contin)
        loss = 0
        for i in range(36):
            loss = loss + abs(curve[i]-onecone[i])

        total_loss = loss.item()

        return total_loss

    def loss_function_d(arg):

        mu, variance, scale, contin= arg
        onecone = cone_norm_model(mu, variance, scale, contin)
        loss = 0
        for i in range(36):
            loss = loss + abs(curve[i+36]-onecone[i])

        total_loss = loss.item()

        return total_loss

    space_norm = [hp.uniform('mu',0, 36), hp.uniform('variance',0, 36), hp.uniform('scale',0, 100), hp.uniform('contin',0, 10)]
    best_norm_c = fmin(loss_function_c, space_norm, algo=tpe.suggest, max_evals = 2000)
    best_norm_d = fmin(loss_function_d, space_norm, algo=tpe.suggest, max_evals = 2000)
    total_loss = loss_function_c(space_eval(space_norm, best_norm_c))+loss_function_d(space_eval(space_norm, best_norm_d))
    return total_loss

100%|██████████| 2000/2000 [00:29<00:00, 68.15trial/s, best loss: 27.110016675435965]
